In [1]:
import tensorflow as tf
import numpy
import data
import model

learning_rate = .001
grad_clip = 5.
batch_size = 64

Reading files: 08527   
Reading successful!


In [2]:
tf.reset_default_graph()
c = model.classifier(rnn_sizes=[128, 128], fc_sizes=[64, 32], batch_size=batch_size)
feeder = data.batch_pool(batch_size)
batch = next(feeder)

# Loss, accuracy and optimizer

In [3]:
'''
    Name scope is good for graph definition for debugging in TensorBoard
'''
labels = tf.placeholder(tf.int64, [c.batch_size], 'labels')
global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')

with tf.name_scope('cross_entropy'):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=c.logits)
    with tf.name_scope('total'):
        loss = tf.reduce_mean(loss)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(c.preds, 1), labels)
    with tf.name_scope('acc'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    gvs = optimizer.compute_gradients(loss)
    with tf.name_scope('gradient_clipping'):
        capped_gvs = [(tf.clip_by_value(grad, -grad_clip, grad_clip), var) 
                      for grad, var in gvs]
        
    opt = optimizer.apply_gradients(capped_gvs, global_step)

# Summaries

In [11]:
summaries = tf.summary.merge([
    [(tf.summary.histogram('gradient', grad), 
      tf.summary.histogram('variable', var)) for grad, var in gvs],
    tf.summary.scalar('loss', loss),
    tf.summary.scalar('accuracy', accuracy)
])

# Train test

In [12]:
sess = tf.InteractiveSession()
writer = tf.summary.FileWriter('/tmp/tensorflow_logdir/', graph=sess.graph)
saver = tf.train.Saver(keep_checkpoint_every_n_hours=1)
tf.global_variables_initializer().run()

In [ ]:
for i, batch in enumerate(feeder):
    state = sess.run(c.zero_state)
    x_feed, label_feed, lens_feed = batch
    
    for idx in range(0, lens_feed.max(), c.time_steps):
        data_window = x_feed[:, idx:idx+c.time_steps]
        lens_window = lens_feed-idx
        
        feed_dict = {
            c.x:data_window,
            c.init_state:state,
            c.seq_len:lens_window,
            labels:label_feedmodel
        }
        
        fetch_dict = {
            'opt' : opt,
            'state' : c.rnn_last_states,
            'step' : global_step,
            'loss' : loss,
            'acc' : accuracy,
            'sum' : summaries
        }
        
        fetched = sess.run(fetch_dict, feed_dict)
        state = fetched['state']
        if idx%(2 * c.time_steps) == 0:
            print('\r%d'%idx, end='', flush=True)
            writer.add_summary(summary=fetched['sum'], global_step=fetched['step'])
    if i%100 == 0: 
        saver.save(sess, c.get_checkpoint_path(), fetched['step'])
    print('\t', i, fetched['loss'], fetched['acc'])
        

18000	 0 0.810768 0.640625
17400	 1 0.817352 0.6875
17400	 2 0.906504 0.515625
17400	 3 0.835465 0.65625
17400	 4 0.800106 0.6875
17400	 5 0.955955 0.578125
17400	 6 0.942971 0.53125
17400	 7 0.725384 0.6875
18000	 8 0.768867 0.6875
17400	 9 0.954396 0.5625
2400